In [1]:
# 20 card slots initialized as "unknown"
cards_memory = ["unknown"] * 16


In [2]:
import SerialCom
# Initialize serial communication
serial_com = SerialCom.SerialCom()

def send_command_to_arduino(command):
    """Sends a command to Arduino and waits for 'DONE' response."""
    serial_com.write(command + "\n")  # Send command
    print(f"📡 Sent to Arduino: {command}")

    # Wait for Arduino to acknowledge
    while True:
        response = serial_com.recieve().strip()
        if response == "DONE":
            print(f"✅ Arduino completed: {command}")
            break
        time.sleep(0.1)


COM1 - FabulaTech Virtual Serial Port (COM1)
COM2 - FabulaTech Virtual Serial Port (COM2)


In [3]:
def find_matching_cards():
    """Finds two cards with the same color in memory."""
    color_positions = {}  # Store color locations

    for i, color in enumerate(cards_memory):
        if color not in ["unknown", "deleted"]:
            if color in color_positions:
                return (color_positions[color], i)  # Return matched indices
            color_positions[color] = i  # Store first occurrence

    return None  # No match found


In [4]:
import random

def pick_random_card():
    """Selects a random unknown card."""
    unknown_positions = [i for i in range(16) if cards_memory[i] == "unknown"]
    if not unknown_positions:
        return None  # No unknown cards left
    return random.choice(unknown_positions)


In [5]:
def flip_card(position):
    """Simulates flipping a card by detecting its color and updating memory."""
    print(f"🃏 Flipping card at position {position}...")

    detected_color = detect_color()  # Simulate color detection
    if detected_color is None:
        print("⚠️ Error: No color detected.")
        return False

    cards_memory[position] = detected_color  # Store the detected color
    print(f"🎨 Detected color at position {position}: {detected_color}")
    return True


In [6]:
def calculate_angles(n):
    if not (0 <= n <= 15):
        raise ValueError("Input number must be between 0 and 15")

    # Placeholder for mathematical operations
    angle1 =  50  # Replace with your formula
    angle2 =  30  # Replace with your formula
    angle3 =  100  # Replace with your formula

    return angle1, angle2, angle3

In [7]:

def check_match(pos1, pos2):
    """Simulates checking if two flipped cards match."""
    if cards_memory[pos1] == cards_memory[pos2]:
        print(f"✅ MATCH FOUND! Removing cards at {pos1} and {pos2}")
        angles1 = calculate_angles(pos1)
        angles2 = calculate_angles(pos2)
        send_command_to_arduino(f"REMOVE {angles1[0]},{angles1[1]},{angles1[2]}")
        
        send_command_to_arduino(f"REMOVE {angles2[0]},{angles2[1]},{angles2[2]}")
        cards_memory[pos1] = "deleted"
        cards_memory[pos2] = "deleted"
        return True
    else:
        print(f"❌ No match. Remembering colors.")
        send_command_to_arduino(f"UnFLIP cards at positions {pos1} and {pos2}")
        return False


In [8]:
import sys
import time
sys.path.append("Color detection")  # Ensure path is correct
import color_detection as cd
import cv2
import threading


def release_camera():
    """Closes all OpenCV windows."""
    cv2.destroyAllWindows()
def display_camera():
    """Displays the phone camera feed in a moderate-sized window with a centered ROI box."""
    cv2.namedWindow("Live Camera", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Live Camera", 800, 600)

    while True:
        frame = cd.get_frame()
        if frame is None:
            continue  # Skip if no frame is captured

        # Get frame dimensions
        h, w, _ = frame.shape
        cx, cy = w // 2, h // 2  # Center coordinates

        # Define a centered ROI (100×100 pixels)
        roi_size = 300
        x1, y1 = cx - roi_size // 2, cy - roi_size // 2
        x2, y2 = cx + roi_size // 2, cy + roi_size // 2

        # Draw dynamically centered ROI box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)

        # Display the live camera feed
        cv2.imshow("Live Camera", frame)

        # Press 'Q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    release_camera()



def play_turn():
    """Handles one turn of the game, ensuring the user flips a card before proceeding."""
    print("\n🔄 NEW TURN STARTED!")

    # Step 1: Check if there’s a known match
    match = find_matching_cards()
    if match:
        pos1, pos2 = match
        print(f"🎯 Matching cards found! Flipping {pos1} and {pos2}")
        angles1 = calculate_angles(pos1)
        angles2 = calculate_angles(pos2)
        send_command_to_arduino(f"MOVE {angles1}")
        send_command_to_arduino(f"FLIP")
        send_command_to_arduino(f"MOVE {angles1}")

        send_command_to_arduino(f"MOVE {angles2}")
        send_command_to_arduino(f"FLIP")
        send_command_to_arduino(f"MOVE {angles2}")
        
        check_match(pos1, pos2)
        return

    # Step 2: Pick a random unknown card
    pos1 = pick_random_card()
    if pos1 is None:
        print("🏁 No more unknown cards left.")
        return
    angles1 = calculate_angles(pos1) # calculate the required angles
    send_command_to_arduino(f"MOVE {angles1[0]},{angles1[1]},{angles1[2]}")
    send_command_to_arduino(f"DETECT") 
    # Step 4: Detect color using the camera module
    detected_color = cd.detect_color()
    print(f"🎨 Detected color at {pos1}: {detected_color}")

    
    
    # Step 5: Save detected color in memory
    cards_memory[pos1] = detected_color
    continue_game = True

    send_command_to_arduino(f"FLIP")
    send_command_to_arduino(f"MOVE {angles1[0]},{angles1[1]},{angles1[2]}")
    # Step 6: Look for a known match
    for i in range(16):
        if cards_memory[i] == cards_memory[pos1] and i != pos1:
            print(f"🎯 Matching found for {cards_memory[pos1]} at {pos1} and {i}")
            angles1 = calculate_angles(i)
           
            send_command_to_arduino(f"MOVE {angles1[0]},{angles1[1]},{angles1[2]}")

            send_command_to_arduino(f"FLIP")
            send_command_to_arduino(f"MOVE {angles1[0]},{angles1[1]},{angles1[2]}")
            continue_game = False
            check_match(pos1, i)
            return
    if continue_game:
        
    # Step 7: Pick a second random card
        pos2 = pick_random_card()
        if pos2 is None:
            print("🏁 No more unknown cards left.")
            return
        angles1 = calculate_angles(pos2) # calculate the required angles
        send_command_to_arduino(f"MOVE {angles1[0]},{angles1[1]},{angles1[2]}")
        send_command_to_arduino(f"DETECT") 
       
        detected_color = cd.detect_color()
        print(f"🎨 Detected color at {pos2}: {detected_color}")
    
    # Step 8: Save detected color in memory
        cards_memory[pos2] = detected_color

        send_command_to_arduino(f"FLIP")
        send_command_to_arduino(f"MOVE {angles1[0]},{angles1[1]},{angles1[2]}")
    # Step 9: Check for a match
        check_match(pos1, pos2)

    print("📝 Current memory:", cards_memory)  # Show memory updates


In [ ]:
camera_thread = threading.Thread(target=display_camera, daemon=True)
camera_thread.start()
while "unknown" in cards_memory:
    play_turn()
    time.sleep(2)


🔄 NEW TURN STARTED!
📡 Sent to Arduino: MOVE 50,30,100
✅ Arduino completed: MOVE 50,30,100
📡 Sent to Arduino: DETECT
✅ Arduino completed: DETECT
🎨 Detected color at 5: ('Black', (np.int64(0), np.int64(0), np.int64(1)))
📡 Sent to Arduino: FLIP
✅ Arduino completed: FLIP
📡 Sent to Arduino: MOVE 50,30,100
✅ Arduino completed: MOVE 50,30,100
📡 Sent to Arduino: MOVE 50,30,100
✅ Arduino completed: MOVE 50,30,100
📡 Sent to Arduino: DETECT
✅ Arduino completed: DETECT
🎨 Detected color at 3: ('Black', (np.int64(0), np.int64(0), np.int64(1)))
📡 Sent to Arduino: FLIP
✅ Arduino completed: FLIP
📡 Sent to Arduino: MOVE 50,30,100
✅ Arduino completed: MOVE 50,30,100
✅ MATCH FOUND! Removing cards at 5 and 3
📡 Sent to Arduino: REMOVE 50,30,100
✅ Arduino completed: REMOVE 50,30,100
📡 Sent to Arduino: REMOVE 50,30,100
✅ Arduino completed: REMOVE 50,30,100
📝 Current memory: ['unknown', 'unknown', 'unknown', 'deleted', 'unknown', 'deleted', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'un

MOVE -> angles
FLIP
DETECT
REMOVE -> angles
